In [1]:
import findspark
findspark.init('/home/omotade/spark-3.3.1-bin-hadoop3/')

In [2]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as pysum, mean, min as pymin, max as pymax 

In [8]:
from scraping import scrape
from pyspark.sql.functions import explode, split, col, lit

In [4]:
spark = SparkSession.builder.\
                            appName("TwitterCovidTweets").\
                            config("spark.mongodb.input.uri", "mongodb://127.0.0.1/test.coll").\
                            config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector:10.0.0').\
                            config("spark.mongodb.output.uri", "mongodb://127.0.0.1/test.test").\
                            getOrCreate()

23/01/17 11:47:35 WARN Utils: Your hostname, DESKTOP-SF4M4SF resolves to a loopback address: 127.0.1.1; using 172.21.224.1 instead (on interface eth2)
23/01/17 11:47:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/omotade/spark-3.3.1-bin-hadoop3/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/omotade/.ivy2/cache
The jars for the packages stored in: /home/omotade/.ivy2/jars
org.mongodb.spark#mongo-spark-connector added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7142e394-c4ce-4614-b24e-8c43620b17ee;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector;10.0.0 in central
You probably access the destination server through a proxy server that is not well configured.
You probably access the destination server through a proxy server that is not well configured.
You probably access the destination server through a proxy server that is not well configured.
	found org.mongodb#mongodb-driver-sync;4.5.1 in central
	[4.5.1] org.mongodb#mongodb-driver-sync;[4.5.0,4.5.99)
	found org.mongodb#bson;4.5.1 in central
	found org.mongodb#mongodb-driver-core;4.5.1 in central
:: resolution report :: resolve 55277ms :: artifacts dl 94ms
	:: modules in use:
	org.mongodb#bson;4.5.1 from central in [default]
	org.mong

23/01/17 11:48:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [32]:
def case_count():
    return scrape()['cases']


spark.udf.register('get_case_count', case_count)

23/01/17 17:35:38 WARN SimpleFunctionRegistry: The function get_case_count replaced a previously registered function.


<function __main__.case_count()>

In [6]:
stream = spark.readStream.format('socket').\
                        option('host', 'localhost').\
                        option('port', '5555').\
                        option('includeTimestamp', True).\
                        load()

23/01/17 11:54:52 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [7]:
print("Streaming Status:", stream.isStreaming)

True

In [39]:

data = stream.select(split(stream.value, 'EOL').alias('content'), (stream.timestamp).alias('timestamp'))
data = data.withColumn('total_case_count', lit(case_count()))

In [46]:
query = data.writeStream\
    .format("mongodb")\
    .option("checkpointLocation", "/tmp/pyspark7/")\
    .option("forceDeleteTempCheckpointLocation", "true")\
    .option('spark.mongodb.connection.uri', "mongodb://127.0.0.1/test")\
    .option('spark.mongodb.database', 'test')\
    .option('spark.mongodb.collection', 'test')\
    .trigger(processingTime = '20 seconds')\
    .outputMode("append")\
    .start()

query.awaitTermination()

23/01/17 18:04:39 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/01/17 18:04:40 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
23/01/17 18:04:40 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.
23/01/17 18:04:40 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
23/01/17 18:04:40 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.
23/01/17 18:04:40 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
23/01/17 18:04:40 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.
23/01/17 18:05:00 WARN CaseInsensitiveStringMap: Converting d

23/01/17 18:09:00 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
23/01/17 18:09:00 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.
23/01/17 18:09:00 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
23/01/17 18:09:00 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.
23/01/17 18:09:00 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
23/01/17 18:09:00 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.
23/01/17 18:09:20 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
23/01/17 18:09:20 WARN CaseInsensitiveStringMap: Converting duplicated key force

23/01/18 10:54:40 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
23/01/18 10:54:40 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.
23/01/18 10:54:40 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
23/01/18 10:54:40 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.
23/01/18 10:54:40 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
23/01/18 10:54:40 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.
23/01/18 10:55:00 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
23/01/18 10:55:00 WARN CaseInsensitiveStringMap: Converting duplicated key force

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/omotade/spark-3.3.1-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/omotade/spark-3.3.1-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/omotade/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [40]:
query = data.writeStream\
    .format("console")\
    .option("forceDeleteTempCheckpointLocation", "true")\
    .trigger(processingTime = '20 seconds')\
    .outputMode("append")\
    .start()

query.awaitTermination()

23/01/17 17:54:44 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-b09e1475-fdbe-49f1-8473-2f6d2b2bc367. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/01/17 17:54:44 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
-------------------------------------------
Batch: 0
-------------------------------------------
+-------+---------+----------------+
|content|timestamp|total_case_count|
+-------+---------+----------------+
+-------+---------+----------------+

-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+--------------------+----------------+
|             content|           timestamp|total_case_count|
+-----

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/omotade/spark-3.3.1-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/omotade/spark-3.3.1-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/omotade/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 